### Initialize

In [32]:
import ee
import csv

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize(project='ee-nazrerukmini')

##COMBINED RGB + MASK SVM

In [33]:
def process_point_svm(lon, lat, city, state, year, filename):
    # Define the center point
    center = ee.Geometry.Point([lon, lat])

    # Define the bounding box with a buffer radius (2560 meters for 512x512 pixels at 10m resolution)
    geometry = center.buffer(2560).bounds()

    # Load Sentinel-2 MSI Level-2A as ImageCollection
    sentinel2 = (ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
                 .filterBounds(geometry)
                 .filterDate(f'{year}-07-01', f'{year}-09-30')
                 .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 3))
                 .median())

    # Calculate indices and masks
    ndvi = sentinel2.normalizedDifference(['B8', 'B4']).rename('NDVI')
    # ndwi = sentinel2.normalizedDifference(['B3', 'B8']).rename('NDWI')
    swm = sentinel2.expression(
        '(B2 + B3) / (B8 + B11)',
        {
            'B2': sentinel2.select('B2'),
            'B3': sentinel2.select('B3'),
            'B8': sentinel2.select('B8'),
            'B11': sentinel2.select('B11')
        }
    ).rename('SWM')

    # Combine NDVI and NDWI into a single image for sampling
    feature_image = ndvi.addBands(swm)

    # Define class masks
    water_mask = swm.gt(1.5)
    building_mask = ndvi.gt(0.0).And(ndvi.lt(0.4))
    sparse_veg_mask = ndvi.gte(0.4).And(ndvi.lt(0.7))
    dense_veg_mask = ndvi.gte(0.7)

    # Sample points for each class
    NUM_OF_PIXELS = 50000

    def sample_points(mask, class_value):
        return feature_image.updateMask(mask).sample(
            region=geometry,
            scale=10,
            numPixels=NUM_OF_PIXELS,
            geometries=True
        ).map(lambda f: f.set('class', class_value))

    water_points = sample_points(water_mask, 0)
    building_points = sample_points(building_mask, 1)
    sparse_veg_points = sample_points(sparse_veg_mask, 2)
    dense_veg_points = sample_points(dense_veg_mask, 3)

    # Merge all training points
    training_points = water_points.merge(building_points).merge(sparse_veg_points).merge(dense_veg_points)

    # Split the data into training (80%) and validation (20%) sets
    with_random = training_points.randomColumn()
    split = 0.8
    training_data = with_random.filter(ee.Filter.lt('random', split))
    validation_data = with_random.filter(ee.Filter.gte('random', split))

    # Train an SVM classifier
    svm_classifier = ee.Classifier.libsvm().train(
        features=training_data,
        classProperty='class',
        inputProperties=['NDVI', 'SWM']
    )

    # Classify the image
    classified_image = feature_image.classify(svm_classifier)
    # classified_image = classified_image.toUint16()

    # # Evaluate the classifier using the validation data
    # validation = validation_data.classify(svm_classifier)
    # validation_confusion_matrix = validation.errorMatrix('class', 'classification')
    # print('Validation Error Matrix:', validation_confusion_matrix.getInfo())
    # print('Validation Overall Accuracy:', validation_confusion_matrix.accuracy().getInfo())

    # # Compute training accuracy
    # training_confusion_matrix = svm_classifier.confusionMatrix()
    # print('Training Error Matrix:', training_confusion_matrix.getInfo())
    # print('Training Overall Accuracy:', training_confusion_matrix.accuracy().getInfo())


    # Evaluate the classifier using the validation data
    validation = validation_data.classify(svm_classifier)
    validation_confusion_matrix = validation.errorMatrix('class', 'classification')
    validation_accuracy = validation_confusion_matrix.accuracy().getInfo()

    # Compute training accuracy
    training_confusion_matrix = svm_classifier.confusionMatrix()
    training_accuracy = training_confusion_matrix.accuracy().getInfo()

    # Save results to CSV
    csv_filename = "classification_results.csv"
    header = ["City", "State", "Year", "Filename",
              "Training Accuracy", "Training Confusion Matrix",
              "Validation Accuracy", "Validation Confusion Matrix"]

    data = [
        city, state, year, filename,
        training_accuracy, training_confusion_matrix.getInfo(),
        validation_accuracy, validation_confusion_matrix.getInfo()
    ]

    # Write to CSV (append if file exists, otherwise create)
    try:
        with open(csv_filename, 'a', newline='') as file:
            writer = csv.writer(file)
            if file.tell() == 0:  # File is empty, write header
                writer.writerow(header)
            writer.writerow(data)
    except Exception as e:
        print(f"Error writing to CSV: {e}")

    print(f"Results saved to {csv_filename}.")


    # Enhanced RGB export
    enhanced_rgb = sentinel2.select(['B4', 'B3', 'B2']).multiply(2.0)
    viz_params = {
        'bands': ['B4', 'B3', 'B2'],
        'min': 0,
        'max': 3000,
        'gamma': 1
    }

    # Export tasks
    # 1. Enhanced RGB
    enhanced_rgb_task = ee.batch.Export.image.toDrive(
        image=enhanced_rgb.uint16().clip(geometry).visualize(**viz_params),
        description=f'EnhancedRGB_{filename}_{year}',
        folder='ChangeDet',
        fileNamePrefix=f'{filename}_{year}',
        region=geometry,
        fileFormat='GEOTIFF',
        crs='EPSG:3857',
        dimensions='512x512'
    )

    # 2. Classified Image
    class_vis = {
        'min': 0,
        'max': 3,
        'palette': ['lightblue', 'white', 'lightgreen', 'darkgreen']
    }
    mask_task = ee.batch.Export.image.toDrive(
        image=classified_image.uint16(),
        description=f'm_{filename}_{year}',
        folder='ChangeDet',
        fileNamePrefix=f'm_{filename}_{year}',
        region=geometry,
        fileFormat='GEOTIFF',
        crs='EPSG:3857',
        dimensions='512x512'
    )

    # Start both export tasks
    enhanced_rgb_task.start()
    mask_task.start()

    print(f"Tasks started for {city} at ({lon}, {lat}).")

### RUN

In [34]:
import io
import pandas as pd
data = pd.read_csv(io.StringIO('''
Arizona,Chandler,2019,2024,"-111.848077176178,33.3049802596002",33.30498026,-111.8480772,"-111.848077176178,33.3549802596002","-111.798077176178,33.3549802596002","-111.798077176178,33.3049802596002","-111.798077176178,33.2549802596002","-111.848077176178,33.2549802596002","-111.898077176178,33.2549802596002","-111.898077176178,33.3049802596002","-111.898077176178,33.3549802596002"
Arizona,Phoenix,2019,2024,"-112.070424160813,33.4481783649878",33.44817836,-112.0704242,"-112.070424160813,33.4981783649878","-112.020424160813,33.4981783649878","-112.020424160813,33.4481783649878","-112.020424160813,33.3981783649878","-112.070424160813,33.3981783649878","-112.120424160813,33.3981783649878","-112.120424160813,33.4481783649878","-112.120424160813,33.4981783649878"
Arizona,Tucson,2019,2024,"-110.870452190922,32.1668049754518",32.16680498,-110.8704522,"-110.870452190922,32.2168049754518","-110.820452190922,32.2168049754518","-110.820452190922,32.1668049754518","-110.820452190922,32.1168049754518","-110.870452190922,32.1168049754518","-110.920452190922,32.1168049754518","-110.920452190922,32.1668049754518","-110.920452190922,32.2168049754518"
California,Bakersfield,2019,2024,"-119.028537732728,35.3756177333075",35.37561773,-119.0285377,"-119.028537732728,35.4256177333075","-118.978537732728,35.4256177333075","-118.978537732728,35.3756177333075","-118.978537732728,35.3256177333075","-119.028537732728,35.3256177333075","-119.078537732728,35.3256177333075","-119.078537732728,35.3756177333075","-119.078537732728,35.4256177333075"
California,Fresno,2019,2024,"-119.789986310795,36.7726306470873",36.77263065,-119.7899863,"-119.789986310795,36.8226306470873","-119.739986310795,36.8226306470873","-119.739986310795,36.7726306470873","-119.739986310795,36.7226306470873","-119.789986310795,36.7226306470873","-119.839986310795,36.7226306470873","-119.839986310795,36.7726306470873","-119.839986310795,36.8226306470873"
California,LA,2019,2024,"-118.264852569023,34.0564551357433",34.05645514,-118.2648526,"-118.264852569023,34.1064551357433","-118.214852569023,34.1064551357433","-118.214852569023,34.0564551357433","-118.214852569023,34.0064551357433","-118.264852569023,34.0064551357433","-118.314852569023,34.0064551357433","-118.314852569023,34.0564551357433","-118.314852569023,34.1064551357433"
California,Sacramento,2019,2024,"-121.502079473134,38.5773276498094",38.57732765,-121.5020795,"-121.502079473134,38.6273276498094","-121.452079473134,38.6273276498094","-121.452079473134,38.5773276498094","-121.452079473134,38.5273276498094","-121.502079473134,38.5273276498094","-121.552079473134,38.5273276498094","-121.552079473134,38.5773276498094","-121.552079473134,38.6273276498094"
California,SanDiego,2019,2024,"-117.075958147392,32.7576576579701",32.75765766,-117.0759581,"-117.075958147392,32.8076576579701","-117.025958147392,32.8076576579701","-117.025958147392,32.7576576579701","-117.025958147392,32.7076576579701","-117.075958147392,32.7076576579701","-117.125958147392,32.7076576579701","-117.125958147392,32.7576576579701","-117.125958147392,32.8076576579701"
California,SanFrancisco,2019,2024,"-122.447559212022,37.7587570417743",37.75875704,-122.4475592,"-122.447559212022,37.8087570417743","-122.397559212022,37.8087570417743","-122.397559212022,37.7587570417743","-122.397559212022,37.7087570417743","-122.447559212022,37.7087570417743","-122.497559212022,37.7087570417743","-122.497559212022,37.7587570417743","-122.497559212022,37.8087570417743"
California,SanJose,2019,2024,"-121.899180305122,37.3391025913052",37.33910259,-121.8991803,"-121.899180305122,37.3891025913052","-121.849180305122,37.3891025913052","-121.849180305122,37.3391025913052","-121.849180305122,37.2891025913052","-121.899180305122,37.2891025913052","-121.949180305122,37.2891025913052","-121.949180305122,37.3391025913052","-121.949180305122,37.3891025913052"
Colorado,ColoradoSprings,2019,2024,"-104.781472477423,38.872598709053",38.87259871,-104.7814725,"-104.781472477423,38.922598709053","-104.731472477423,38.922598709053","-104.731472477423,38.872598709053","-104.731472477423,38.822598709053","-104.781472477423,38.822598709053","-104.831472477423,38.822598709053","-104.831472477423,38.872598709053","-104.831472477423,38.922598709053"
Colorado,Denver,2019,2024,"-104.989745192253,39.7378361614133",39.73783616,-104.9897452,"-104.989745192253,39.7878361614133","-104.939745192253,39.7878361614133","-104.939745192253,39.7378361614133","-104.939745192253,39.6878361614133","-104.989745192253,39.6878361614133","-105.039745192253,39.6878361614133","-105.039745192253,39.7378361614133","-105.039745192253,39.7878361614133"
Colorado,FortCollins,2019,2024,"-105.074864515804,40.5534989764619",40.55349898,-105.0748645,"-105.074864515804,40.6034989764619","-105.024864515804,40.6034989764619","-105.024864515804,40.5534989764619","-105.024864515804,40.5034989764619","-105.074864515804,40.5034989764619","-105.124864515804,40.5034989764619","-105.124864515804,40.5534989764619","-105.124864515804,40.6034989764619"
Idaho,Boise,2019,2024,"-116.223600922482,43.6087318201907",43.60873182,-116.2236009,"-116.223600922482,43.6587318201907","-116.173600922482,43.6587318201907","-116.173600922482,43.6087318201907","-116.173600922482,43.5587318201907","-116.223600922482,43.5587318201907","-116.273600922482,43.5587318201907","-116.273600922482,43.6087318201907","-116.273600922482,43.6587318201907"
Idaho,IdahoFalls,2019,2024,"-112.035124191445,43.4898874704146",43.48988747,-112.0351242,"-112.035124191445,43.5398874704146","-111.985124191445,43.5398874704146","-111.985124191445,43.4898874704146","-111.985124191445,43.4398874704146","-112.035124191445,43.4398874704146","-112.085124191445,43.4398874704146","-112.085124191445,43.4898874704146","-112.085124191445,43.5398874704146"
Idaho,Sandpoint,2019,2024,"-116.563324237704,48.2795150792751",48.27951508,-116.5633242,"-116.563324237704,48.3295150792751","-116.513324237704,48.3295150792751","-116.513324237704,48.2795150792751","-116.513324237704,48.2295150792751","-116.563324237704,48.2295150792751","-116.613324237704,48.2295150792751","-116.613324237704,48.2795150792751","-116.613324237704,48.3295150792751"
Idaho,SunValley,2019,2024,"-114.336953602359,43.6812400973637",43.6812401,-114.3369536,"-114.336953602359,43.7312400973637","-114.286953602359,43.7312400973637","-114.286953602359,43.6812400973637","-114.286953602359,43.6312400973637","-114.336953602359,43.6312400973637","-114.386953602359,43.6312400973637","-114.386953602359,43.6812400973637","-114.386953602359,43.7312400973637"
Montana,Bozeman,2019,2024,"-111.062843483358,45.6822407656587",45.68224077,-111.0628435,"-111.062843483358,45.7322407656587","-111.012843483358,45.7322407656587","-111.012843483358,45.6822407656587","-111.012843483358,45.6322407656587","-111.062843483358,45.6322407656587","-111.112843483358,45.6322407656587","-111.112843483358,45.6822407656587","-111.112843483358,45.7322407656587"
Montana,Flathead,2019,2024,"-114.166456018661,48.4223478398922",48.42234784,-114.166456,"-114.166456018661,48.4723478398922","-114.116456018661,48.4723478398922","-114.116456018661,48.4223478398922","-114.116456018661,48.3723478398922","-114.166456018661,48.3723478398922","-114.216456018661,48.3723478398922","-114.216456018661,48.4223478398922","-114.216456018661,48.4723478398922"
Montana,Glasgow,2019,2024,"-106.634196061284,48.1982038089386",48.19820381,-106.6341961,"-106.634196061284,48.2482038089386","-106.584196061284,48.2482038089386","-106.584196061284,48.1982038089386","-106.584196061284,48.1482038089386","-106.634196061284,48.1482038089386","-106.684196061284,48.1482038089386","-106.684196061284,48.1982038089386","-106.684196061284,48.2482038089386"
Montana,GreatFalls,2019,2024,"-111.294934254734,47.5052242841823",47.50522428,-111.2949343,"-111.294934254734,47.5552242841823","-111.244934254734,47.5552242841823","-111.244934254734,47.5052242841823","-111.244934254734,47.4552242841823","-111.294934254734,47.4552242841823","-111.344934254734,47.4552242841823","-111.344934254734,47.5052242841823","-111.344934254734,47.5552242841823"
Nevada,CarsonCity,2019,2024,"-119.751204166269,39.1633568752798",39.16335688,-119.7512042,"-119.751204166269,39.2133568752798","-119.701204166269,39.2133568752798","-119.701204166269,39.1633568752798","-119.701204166269,39.1133568752798","-119.751204166269,39.1133568752798","-119.801204166269,39.1133568752798","-119.801204166269,39.1633568752798","-119.801204166269,39.2133568752798"
Nevada,LasVegas,2019,2024,"-115.247956672823,36.189853881388",36.18985388,-115.2479567,"-115.247956672823,36.239853881388","-115.197956672823,36.239853881388","-115.197956672823,36.189853881388","-115.197956672823,36.139853881388","-115.247956672823,36.139853881388","-115.297956672823,36.139853881388","-115.297956672823,36.189853881388","-115.297956672823,36.239853881388"
Nevada,Reno,2019,2024,"-119.808923098678,39.5294897798833",39.52948978,-119.8089231,"-119.808923098678,39.5794897798833","-119.758923098678,39.5794897798833","-119.758923098678,39.5294897798833","-119.758923098678,39.4794897798833","-119.808923098678,39.4794897798833","-119.858923098678,39.4794897798833","-119.858923098678,39.5294897798833","-119.858923098678,39.5794897798833"
NewMexico,Albuquerque,2019,2024,"-106.659297259854,35.1133063463688",35.11330635,-106.6592973,"-106.659297259854,35.1633063463688","-106.609297259854,35.1633063463688","-106.609297259854,35.1133063463688","-106.609297259854,35.0633063463688","-106.659297259854,35.0633063463688","-106.709297259854,35.0633063463688","-106.709297259854,35.1133063463688","-106.709297259854,35.1633063463688"
NewMexico,LasCruces,2019,2024,"-106.767400513854,32.3234919524856",32.32349195,-106.7674005,"-106.767400513854,32.3734919524856","-106.717400513854,32.3734919524856","-106.717400513854,32.3234919524856","-106.717400513854,32.2734919524856","-106.767400513854,32.2734919524856","-106.817400513854,32.2734919524856","-106.817400513854,32.3234919524856","-106.817400513854,32.3734919524856"
NewMexico,SantaFe,2019,2024,"-105.949181777166,35.6812867375285",35.68128674,-105.9491818,"-105.949181777166,35.7312867375285","-105.899181777166,35.7312867375285","-105.899181777166,35.6812867375285","-105.899181777166,35.6312867375285","-105.949181777166,35.6312867375285","-105.999181777166,35.6312867375285","-105.999181777166,35.6812867375285","-105.999181777166,35.7312867375285"
Oregon,Eugene,2019,2024,"-123.086160872783,44.0512599467363",44.05125995,-123.0861609,"-123.086160872783,44.1012599467363","-123.036160872783,44.1012599467363","-123.036160872783,44.0512599467363","-123.036160872783,44.0012599467363","-123.086160872783,44.0012599467363","-123.136160872783,44.0012599467363","-123.136160872783,44.0512599467363","-123.136160872783,44.1012599467363"
Oregon,Gresham,2019,2024,"-122.433947858986,45.5052587837414",45.50525878,-122.4339479,"-122.433947858986,45.5552587837414","-122.383947858986,45.5552587837414","-122.383947858986,45.5052587837414","-122.383947858986,45.4552587837414","-122.433947858986,45.4552587837414","-122.483947858986,45.4552587837414","-122.483947858986,45.5052587837414","-122.483947858986,45.5552587837414"
Oregon,Hillsboro,2019,2024,"-122.940158612656,45.5294371013982",45.5294371,-122.9401586,"-122.940158612656,45.5794371013982","-122.890158612656,45.5794371013982","-122.890158612656,45.5294371013982","-122.890158612656,45.4794371013982","-122.940158612656,45.4794371013982","-122.990158612656,45.4794371013982","-122.990158612656,45.5294371013982","-122.990158612656,45.5794371013982"
Oregon,Portland,2019,2024,"-122.67348106146,45.5125030786011",45.51250308,-122.6734811,"-122.67348106146,45.5625030786011","-122.62348106146,45.5625030786011","-122.62348106146,45.5125030786011","-122.62348106146,45.4625030786011","-122.67348106146,45.4625030786011","-122.72348106146,45.4625030786011","-122.72348106146,45.5125030786011","-122.72348106146,45.5625030786011"
Oregon,Salem,2019,2024,"-123.033703108334,44.9431690052168",44.94316901,-123.0337031,"-123.033703108334,44.9931690052168","-122.983703108334,44.9931690052168","-122.983703108334,44.9431690052168","-122.983703108334,44.8931690052168","-123.033703108334,44.8931690052168","-123.083703108334,44.8931690052168","-123.083703108334,44.9431690052168","-123.083703108334,44.9931690052168"
Utah,Provo,2019,2024,"-111.656764133701,40.2387006404322",40.23870064,-111.6567641,"-111.656764133701,40.2887006404322","-111.606764133701,40.2887006404322","-111.606764133701,40.2387006404322","-111.606764133701,40.1887006404322","-111.656764133701,40.1887006404322","-111.706764133701,40.1887006404322","-111.706764133701,40.2387006404322","-111.706764133701,40.2887006404322"
Utah,SaltLakeCity,2019,2024,"-111.902402328545,40.7607548377698",40.76075484,-111.9024023,"-111.902402328545,40.8107548377698","-111.852402328545,40.8107548377698","-111.852402328545,40.7607548377698","-111.852402328545,40.7107548377698","-111.902402328545,40.7107548377698","-111.952402328545,40.7107548377698","-111.952402328545,40.7607548377698","-111.952402328545,40.8107548377698"
Utah,StGeorge,2019,2024,"-113.569680239017,37.0922089732",37.09220897,-113.5696802,"-113.569680239017,37.1422089732","-113.519680239017,37.1422089732","-113.519680239017,37.0922089732","-113.519680239017,37.0422089732","-113.569680239017,37.0422089732","-113.619680239017,37.0422089732","-113.619680239017,37.0922089732","-113.619680239017,37.1422089732"
Washington,Olympia,2019,2024,"-122.904596496186,47.0366492043413",47.0366492,-122.9045965,"-122.904596496186,47.0866492043413","-122.854596496186,47.0866492043413","-122.854596496186,47.0366492043413","-122.854596496186,46.9866492043413","-122.904596496186,46.9866492043413","-122.954596496186,46.9866492043413","-122.954596496186,47.0366492043413","-122.954596496186,47.0866492043413"
Washington,Seattle,2019,2024,"-122.330240289355,47.6067060125638",47.60670601,-122.3302403,"-122.330240289355,47.6567060125638","-122.280240289355,47.6567060125638","-122.280240289355,47.6067060125638","-122.280240289355,47.5567060125638","-122.330240289355,47.5567060125638","-122.380240289355,47.5567060125638","-122.380240289355,47.6067060125638","-122.380240289355,47.6567060125638"
Washington,Wenatchee,2019,2024,"-120.334508797937,47.4285902674532",47.42859027,-120.3345088,"-120.334508797937,47.4785902674532","-120.284508797937,47.4785902674532","-120.284508797937,47.4285902674532","-120.284508797937,47.3785902674532","-120.334508797937,47.3785902674532","-120.384508797937,47.3785902674532","-120.384508797937,47.4285902674532","-120.384508797937,47.4785902674532"
Washington,Yakima,2019,2024,"-120.547998145726,46.5943478911759",46.59434789,-120.5479981,"-120.547998145726,46.6443478911759","-120.497998145726,46.6443478911759","-120.497998145726,46.5943478911759","-120.497998145726,46.5443478911759","-120.547998145726,46.5443478911759","-120.597998145726,46.5443478911759","-120.597998145726,46.5943478911759","-120.597998145726,46.6443478911759"
Wyoming,Casper,2019,2024,"-106.345006736291,42.824252387961",42.82425239,-106.3450067,"-106.345006736291,42.874252387961","-106.295006736291,42.874252387961","-106.295006736291,42.824252387961","-106.295006736291,42.774252387961","-106.345006736291,42.774252387961","-106.395006736291,42.774252387961","-106.395006736291,42.824252387961","-106.395006736291,42.874252387961"
Wyoming,Cheyenne,2019,2024,"-104.804142796993,41.1464690202818",41.14646902,-104.8041428,"-104.804142796993,41.1964690202818","-104.754142796993,41.1964690202818","-104.754142796993,41.1464690202818","-104.754142796993,41.0964690202818","-104.804142796993,41.0964690202818","-104.854142796993,41.0964690202818","-104.854142796993,41.1464690202818","-104.854142796993,41.1964690202818"
Arkansas,Fayetteville,2019,2024,"-94.1693611423203,36.0793415239365",36.07934152,-94.16936114,"-94.1693611423203,36.1293415239365","-94.1193611423203,36.1293415239365","-94.1193611423203,36.0793415239365","-94.1193611423203,36.0293415239365","-94.1693611423203,36.0293415239365","-94.2193611423203,36.0293415239365","-94.2193611423203,36.0793415239365","-94.2193611423203,36.1293415239365"
Arkansas,FortSmith,2019,2024,"-94.3761001803224,35.349465145599",35.34946515,-94.37610018,"-94.3761001803224,35.399465145599","-94.3261001803224,35.399465145599","-94.3261001803224,35.349465145599","-94.3261001803224,35.299465145599","-94.3761001803224,35.299465145599","-94.4261001803224,35.299465145599","-94.4261001803224,35.349465145599","-94.4261001803224,35.399465145599"
Arkansas,Jonesboro,2019,2024,"-90.6890316976071,35.8220251075349",35.82202511,-90.6890317,"-90.6890316976071,35.8720251075349","-90.6390316976071,35.8720251075349","-90.6390316976071,35.8220251075349","-90.6390316976071,35.7720251075349","-90.6890316976071,35.7720251075349","-90.7390316976071,35.7720251075349","-90.7390316976071,35.8220251075349","-90.7390316976071,35.8720251075349"
Arkansas,LittleRock,2019,2024,"-92.3587908351279,34.7239202125372",34.72392021,-92.35879084,"-92.3587908351279,34.7739202125372","-92.3087908351279,34.7739202125372","-92.3087908351279,34.7239202125372","-92.3087908351279,34.6739202125372","-92.3587908351279,34.6739202125372","-92.4087908351279,34.6739202125372","-92.4087908351279,34.7239202125372","-92.4087908351279,34.7739202125372"
Iowa,CedarRapids,2019,2024,"-91.6619925526663,41.9778721803251",41.97787218,-91.66199255,"-91.6619925526663,42.0278721803251","-91.6119925526663,42.0278721803251","-91.6119925526663,41.9778721803251","-91.6119925526663,41.9278721803251","-91.6619925526663,41.9278721803251","-91.7119925526663,41.9278721803251","-91.7119925526663,41.9778721803251","-91.7119925526663,42.0278721803251"
Iowa,Davenport,2019,2024,"-90.613404870895,41.5584368357375",41.55843684,-90.61340487,"-90.613404870895,41.6084368357375","-90.563404870895,41.6084368357375","-90.563404870895,41.5584368357375","-90.563404870895,41.5084368357375","-90.613404870895,41.5084368357375","-90.663404870895,41.5084368357375","-90.663404870895,41.5584368357375","-90.663404870895,41.6084368357375"
Iowa,DesMoines,2019,2024,"-93.6264308805809,41.585306640983",41.58530664,-93.62643088,"-93.6264308805809,41.635306640983","-93.5764308805809,41.635306640983","-93.5764308805809,41.585306640983","-93.5764308805809,41.535306640983","-93.6264308805809,41.535306640983","-93.6764308805809,41.535306640983","-93.6764308805809,41.585306640983","-93.6764308805809,41.635306640983"
Kansas,KansasCity,2019,2024,"-94.606290382146,39.0949965578805",39.09499656,-94.60629038,"-94.606290382146,39.1449965578805","-94.556290382146,39.1449965578805","-94.556290382146,39.0949965578805","-94.556290382146,39.0449965578805","-94.606290382146,39.0449965578805","-94.656290382146,39.0449965578805","-94.656290382146,39.0949965578805","-94.656290382146,39.1449965578805"
Kansas,Witchita,2019,2024,"-97.3406433754681,37.6852609509513",37.68526095,-97.34064338,"-97.3406433754681,37.7352609509513","-97.2906433754681,37.7352609509513","-97.2906433754681,37.6852609509513","-97.2906433754681,37.6352609509513","-97.3406433754681,37.6352609509513","-97.3906433754681,37.6352609509513","-97.3906433754681,37.6852609509513","-97.3906433754681,37.7352609509513"
Louisianna,NewOrleans,2019,2024,"-90.1050365909887,29.9497378792779",29.94973788,-90.10503659,"-90.1050365909887,29.9997378792779","-90.0550365909887,29.9997378792779","-90.0550365909887,29.9497378792779","-90.0550365909887,29.8997378792779","-90.1050365909887,29.8997378792779","-90.1550365909887,29.8997378792779","-90.1550365909887,29.9497378792779","-90.1550365909887,29.9997378792779"
Louisianna,Shreveport,2019,2024,"-93.7505570642871,32.5244535763682",32.52445358,-93.75055706,"-93.7505570642871,32.5744535763682","-93.7005570642871,32.5744535763682","-93.7005570642871,32.5244535763682","-93.7005570642871,32.4744535763682","-93.7505570642871,32.4744535763682","-93.8005570642871,32.4744535763682","-93.8005570642871,32.5244535763682","-93.8005570642871,32.5744535763682"
Minnesota,Duluth,2019,2024,"-92.1368185208927,46.8157255924529",46.81572559,-92.13681852,"-92.1368185208927,46.8657255924529","-92.0868185208927,46.8657255924529","-92.0868185208927,46.8157255924529","-92.0868185208927,46.7657255924529","-92.1368185208927,46.7657255924529","-92.1868185208927,46.7657255924529","-92.1868185208927,46.8157255924529","-92.1868185208927,46.8657255924529"
Minnesota,Minneapolis,2019,2024,"-93.2717802211627,44.9750392295275",44.97503923,-93.27178022,"-93.2717802211627,45.0250392295275","-93.2217802211627,45.0250392295275","-93.2217802211627,44.9750392295275","-93.2217802211627,44.9250392295275","-93.2717802211627,44.9250392295275","-93.3217802211627,44.9250392295275","-93.3217802211627,44.9750392295275","-93.3217802211627,45.0250392295275"
Missouri,Columbia,2019,2024,"-92.3332937902474,38.9514620454378",38.95146205,-92.33329379,"-92.3332937902474,39.0014620454378","-92.2832937902474,39.0014620454378","-92.2832937902474,38.9514620454378","-92.2832937902474,38.9014620454378","-92.3332937902474,38.9014620454378","-92.3832937902474,38.9014620454378","-92.3832937902474,38.9514620454378","-92.3832937902474,39.0014620454378"
Missouri,KansasCityMi,2019,2024,"-94.5791616821295,39.0976649961531",39.097665,-94.57916168,"-94.5791616821295,39.1476649961531","-94.5291616821295,39.1476649961531","-94.5291616821295,39.0976649961531","-94.5291616821295,39.0476649961531","-94.5791616821295,39.0476649961531","-94.6291616821295,39.0476649961531","-94.6291616821295,39.0976649961531","-94.6291616821295,39.1476649961531"
Missouri,SaintLouis,2019,2024,"-90.2420909703093,38.6294735220411",38.62947352,-90.24209097,"-90.2420909703093,38.6794735220411","-90.1920909703093,38.6794735220411","-90.1920909703093,38.6294735220411","-90.1920909703093,38.5794735220411","-90.2420909703093,38.5794735220411","-90.2920909703093,38.5794735220411","-90.2920909703093,38.6294735220411","-90.2920909703093,38.6794735220411"
Missouri,SpringfieldMi,2019,2024,"-89.652294146612,39.7820155601036",39.78201556,-89.65229415,"-89.652294146612,39.8320155601036","-89.602294146612,39.8320155601036","-89.602294146612,39.7820155601036","-89.602294146612,39.7320155601036","-89.652294146612,39.7320155601036","-89.702294146612,39.7320155601036","-89.702294146612,39.7820155601036","-89.702294146612,39.8320155601036"
Nebraska,GrandIsland,2019,2024,"-98.3443844263202,40.9273840032397",40.927384,-98.34438443,"-98.3443844263202,40.9773840032397","-98.2943844263202,40.9773840032397","-98.2943844263202,40.9273840032397","-98.2943844263202,40.8773840032397","-98.3443844263202,40.8773840032397","-98.3943844263202,40.8773840032397","-98.3943844263202,40.9273840032397","-98.3943844263202,40.9773840032397"
Nebraska,Omaha,2019,2024,"-95.9384262866491,41.2562509981341",41.256251,-95.93842629,"-95.9384262866491,41.3062509981341","-95.8884262866491,41.3062509981341","-95.8884262866491,41.2562509981341","-95.8884262866491,41.2062509981341","-95.9384262866491,41.2062509981341","-95.9884262866491,41.2062509981341","-95.9884262866491,41.2562509981341","-95.9884262866491,41.3062509981341"
NorthDakota,DevilsLake,2019,2024,"-98.8676195426632,48.1124199680627",48.11241997,-98.86761954,"-98.8676195426632,48.1624199680627","-98.8176195426632,48.1624199680627","-98.8176195426632,48.1124199680627","-98.8176195426632,48.0624199680627","-98.8676195426632,48.0624199680627","-98.9176195426632,48.0624199680627","-98.9176195426632,48.1124199680627","-98.9176195426632,48.1624199680627"
NorthDakota,Fargo,2019,2024,"-96.7921215886654,46.8790792107547",46.87907921,-96.79212159,"-96.7921215886654,46.9290792107547","-96.7421215886654,46.9290792107547","-96.7421215886654,46.8790792107547","-96.7421215886654,46.8290792107547","-96.7921215886654,46.8290792107547","-96.8421215886654,46.8290792107547","-96.8421215886654,46.8790792107547","-96.8421215886654,46.9290792107547"
NorthDakota,GrandForks,2019,2024,"-97.0410598552808,47.9255173074647",47.92551731,-97.04105986,"-97.0410598552808,47.9755173074647","-96.9910598552808,47.9755173074647","-96.9910598552808,47.9255173074647","-96.9910598552808,47.8755173074647","-97.0410598552808,47.8755173074647","-97.0910598552808,47.8755173074647","-97.0910598552808,47.9255173074647","-97.0910598552808,47.9755173074647"
NorthDakota,Jamestown,2019,2024,"-98.7098125579694,46.9101280096508",46.91012801,-98.70981256,"-98.7098125579694,46.9601280096508","-98.6598125579694,46.9601280096508","-98.6598125579694,46.9101280096508","-98.6598125579694,46.8601280096508","-98.7098125579694,46.8601280096508","-98.7598125579694,46.8601280096508","-98.7598125579694,46.9101280096508","-98.7598125579694,46.9601280096508"
NorthDakota,Minot,2019,2024,"-101.293011378648,48.2327688675402",48.23276887,-101.2930114,"-101.293011378648,48.2827688675402","-101.243011378648,48.2827688675402","-101.243011378648,48.2327688675402","-101.243011378648,48.1827688675402","-101.293011378648,48.1827688675402","-101.343011378648,48.1827688675402","-101.343011378648,48.2327688675402","-101.343011378648,48.2827688675402"
NorthDakota,RapidCity,2019,2024,"-103.229271793787,44.0813062819262",44.08130628,-103.2292718,"-103.229271793787,44.1313062819262","-103.179271793787,44.1313062819262","-103.179271793787,44.0813062819262","-103.179271793787,44.0313062819262","-103.229271793787,44.0313062819262","-103.279271793787,44.0313062819262","-103.279271793787,44.0813062819262","-103.279271793787,44.1313062819262"
Oklahoma,Austin,2019,2024,"-97.7454427675468,30.2715934476918",30.27159345,-97.74544277,"-97.7454427675468,30.3215934476918","-97.6954427675468,30.3215934476918","-97.6954427675468,30.2715934476918","-97.6954427675468,30.2215934476918","-97.7454427675468,30.2215934476918","-97.7954427675468,30.2215934476918","-97.7954427675468,30.2715934476918","-97.7954427675468,30.3215934476918"
Oklahoma,Dallas,2019,2024,"-96.8015903906816,32.7721802150102",32.77218022,-96.80159039,"-96.8015903906816,32.8221802150102","-96.7515903906816,32.8221802150102","-96.7515903906816,32.7721802150102","-96.7515903906816,32.7221802150102","-96.8015903906816,32.7221802150102","-96.8515903906816,32.7221802150102","-96.8515903906816,32.7721802150102","-96.8515903906816,32.8221802150102"
Oklahoma,OklahomaCity,2019,2024,"-97.5162763068314,35.4591610814419",35.45916108,-97.51627631,"-97.5162763068314,35.5091610814419","-97.4662763068314,35.5091610814419","-97.4662763068314,35.4591610814419","-97.4662763068314,35.4091610814419","-97.5162763068314,35.4091610814419","-97.5662763068314,35.4091610814419","-97.5662763068314,35.4591610814419","-97.5662763068314,35.5091610814419"
Oklahoma,SanAntonio,2019,2024,"-98.5020168310561,29.4322638038803",29.4322638,-98.50201683,"-98.5020168310561,29.4822638038803","-98.4520168310561,29.4822638038803","-98.4520168310561,29.4322638038803","-98.4520168310561,29.3822638038803","-98.5020168310561,29.3822638038803","-98.5520168310561,29.3822638038803","-98.5520168310561,29.4322638038803","-98.5520168310561,29.4822638038803"
Oklahoma,Tulsa,2019,2024,"-95.9995467029476,36.1528559311182",36.15285593,-95.9995467,"-95.9995467029476,36.2028559311182","-95.9495467029476,36.2028559311182","-95.9495467029476,36.1528559311182","-95.9495467029476,36.1028559311182","-95.9995467029476,36.1028559311182","-96.0495467029476,36.1028559311182","-96.0495467029476,36.1528559311182","-96.0495467029476,36.2028559311182"
'''), header=None)


In [35]:
coord_list = [4, 7, 8, 9, 10, 11, 12, 13, 14]
coord_name = ['C', 'N', 'NE', 'E', 'SE', 'S', 'SW', 'W', 'NW']
ncities = data.shape[0]
# Process each city
for j in range(ncities):
  cityData = data.iloc[j]
  city = cityData[1]
  state = cityData[0]
  # print(cityData)
  print(city, state)
  for i in range(9):
      coord = cityData.iloc[coord_list[i]].split(',')
      filename = f'{state}_{city}_{coord_name[i]}'
      print(filename, end=';')

      lon = float(coord[0])
      lat = float(coord[1])
      print(lon, lat)
      # process_point(lon, lat, city, state, '2019', filename)
      # process_point(lon, lat, city, state, '2024', filename)
      process_point_svm(lon, lat, city, state, '2019', filename)
      process_point_svm(lon, lat, city, state, '2024', filename)
  print()

print("All tasks have been started.")

Chandler Arizona
Arizona_Chandler_C;-111.848077176178 33.3049802596002
Results saved to classification_results.csv.
Tasks started for Chandler at (-111.848077176178, 33.3049802596002).
Results saved to classification_results.csv.
Tasks started for Chandler at (-111.848077176178, 33.3049802596002).
Arizona_Chandler_N;-111.848077176178 33.3549802596002
Results saved to classification_results.csv.
Tasks started for Chandler at (-111.848077176178, 33.3549802596002).
Results saved to classification_results.csv.
Tasks started for Chandler at (-111.848077176178, 33.3549802596002).
Arizona_Chandler_NE;-111.798077176178 33.3549802596002
Results saved to classification_results.csv.
Tasks started for Chandler at (-111.798077176178, 33.3549802596002).
Results saved to classification_results.csv.
Tasks started for Chandler at (-111.798077176178, 33.3549802596002).
Arizona_Chandler_E;-111.798077176178 33.3049802596002
Results saved to classification_results.csv.
Tasks started for Chandler at (-111.7

KeyboardInterrupt: 